# 간단한 데이터 집계와 시각화


### 주요 내용

1. 평균 등 집계값 계산
2. 피벗 테이블 생성 
3. 히스토그램 등 그래프 작성

<br>

### 목표 
1. 건수, 평균 등 집계값을 계산한다.
2. 피벗테이블로 집계값을 계산할 수 있다. 
3. 히스토그램, 막대그래프 등 상황에 맞는 그래프를 작성한다. 


<br>
<hr>
<br>

# 1. 데이터 집계와 시각화

일반적으로 데이터 전처리 이후 평균, 건수 등을 계산하는 집계와 히스토그램과 같은 그래프를 그리는 시각화가 이뤄집니다. 집계와 시각화는 별도의 과정에서 더 자세히 다룰 예정이고, 여기서는 간단한 몇가지 예제를 함께 살펴보겠습니다.

### 1.1. 건수, 평균 등의 계산

먼저 관심 변수를 선택한 다음, *mean()*과 같은 메서드를 활용하여 집계값을 계산할 수 있습니다.  
변수 선택 이전에 *groupby()*를 추가하고 그룹별 집계값 계산도 가능합니다.

In [ ]:
import pandas as pd
df_ins = pd.read_csv('data/insurance.csv')
df_sp  = pd.read_csv('data/StudentsPerformance.csv')

In [ ]:
df_ins['charges'].sum() 
df_ins['charges'].mean()
df_ins['charges'].median()
df_ins['charges'].count()

In [ ]:
# 그룹별 집계값 계산
df_ins.groupby('sex')['charges'].mean()

In [ ]:
# 그룹별 건수 계산
df_ins.groupby('sex')['charges'].count()
df_ins['sex'].value_counts()

In [ ]:
# DataFrame 형식으로 출력
df_ins.groupby('sex', as_index=False)['charges'].mean()

In [ ]:
# 복수 그룹 변수의 활용
df_ins.groupby(['sex', 'smoker'])['charges'].mean()

In [ ]:
# 복수 그룹 변수의 활용(DataFrame 형식으로 출력)
df_ins.groupby(['sex', 'smoker'], as_index=False)['charges'].mean()

In [ ]:
# 복수 수치형 변수의 동일한 집계값 계산
df_ins.groupby(['sex', 'smoker'], as_index=False)[['charges', 'age']].mean()

In [ ]:
# 다양한 변수와 다양한 함수 활용한 집계
# agg() 활용 
agg1 = df_ins.groupby('sex', as_index=False).agg({'charges':['count', 'mean'], 'age':['min', 'max']})
agg1

<br>

#### [실습] df_sp을 활용

1. 'math score'의 평균 계산
2. as_index=True, as_index=False를 활용해서 1.의 결과를 Series, DataFrame 형식으로 바꿔 출력
3. 'race/ethnicity'별 'math score'의 평균 계산
4. 'race/ethnicity', 'lunch' 수준 조합별 'math score', 'reading score', 'writing score'의 평균 계산


In [ ]:
df_sp.head()

### 1.2. 피벗테이블 생성

pandas의 *pivot_table()* 을 활용하면 Excel의 피벗테이블과 동일한 표 형태의 집계가 가능합니다.  

+ index: 행 변수(범주형, 그룹) 지정
+ columns: 열 변수(범주형, 그룹) 지정
+ values: 값 변수 지정
+ aggfunc: 집계함수 지정

In [ ]:
pt1 = df_ins.pivot_table(index='region', columns='smoker', values='charges', aggfunc='mean')
pt1

<br>

#### [실습] df_sp 활용

1. 'race/ethnicity', 'parental level of education'별 'math score'의 평균을 피벗테이블로 계산

### 1.3. 그래프의 작성

Python에서 그래프를 작성할 때 **matplotlib** 라이브러리를 기본적으로 활용할 수 있습니다.  
이 과정에서는 좀 더 직관적으로 코드를 작성하고 그래프를 그릴 수 있는 **seaborn** 라이브러리를 활용합니다.

In [ ]:
import seaborn as sns

In [ ]:
# 히스토그램 작성
sns.histplot(data=df_ins, 
             x='charges')

In [ ]:
# 막대그래프 작성
sns.countplot(data=df_ins, 
              x='region')

In [ ]:
# 피벗 테이블을 열지도로 표현
sns.heatmap(pt1, cmap='Blues', annot=True, fmt='.2f')

<br>

#### [실습] df_sp 활용

1. 'math score'의 히스토그램 생성
2. 'gender'별 학생 수를 계산 및 막대그래프 생성
3. 'race/ethnicity', 'parental level of education'별 'math score'의 평균 피벗테이블을 열지도로 표현

## 2. 데이터 집계와 데이터 처리 

데이터 집계를 통해 얻은 정보를 다양한 방법으로 데이터 처리에 활용할 수 있습니다.  
가장 대표적인 방법은 데이터 집계를 통해 관심 대상을 확인하고 선택하는 방법입니다.

In [ ]:
# 집계값의 계산
agg2 = df_ins.groupby('region', as_index=False)['charges'].median()
agg2

In [ ]:
# 상위 2개 지역 선택
target_region = agg2.nlargest(2, 'charges')['region']
target_region

In [ ]:
# 전체 관측치에서 .isin()을 활용하여 해당 지역 관측치만 선택
cond = df_ins['region'].isin(target_region)
cond

In [ ]:
df_ins[cond]

<br>

#### [실습] df_sp 활용

1. 'parental level of education'별 'reading score'의 평균을 계산
2. 1.의 결과에서 평균 점수 기준 상위 3개 부모교육수준 그룹을 *target*으로 저장하기
3. 'parental level of education'과 `.isin()`, 2.에서 저장한 *target*을 활용하여 전체 데이터에서 해당 그룹 관측치만 선택하고 **df_sub**로 저장
4. 3.의 부분 데이터 **df_sub**를 활용하여 'race/ethnicity'별 'writing score'의 평균 계산

<br>
<hr>
<br>

## 2. 반응형 그래프의 활용

**plotly**를 활용해서 반응형 그래프를 그려서 탐색적 데이터 분석 과정에서 유용하게 활용가능합니다.

* 참고: [Plotly](https://plotly.com/python/)

In [ ]:
# seaborn을 활용한 산점도 
sns.scatterplot(data=df_ins, 
                x='bmi',
                y='charges', 
                hue='smoker')

In [ ]:
import plotly.express as px
px.scatter(df_ins,
           x='bmi',
           y='charges', 
           color='smoker')

#### [실습]
1. df_sp에서 `parental level of education`별로 색을 다르게 한  `math score`와 `reading score`의 산점도 그리기 

#### End of script